<a href="https://colab.research.google.com/github/emanuelpedrozo/ingestao_catalogacao_dados/blob/main/DADOS_EXAMES_DATA_LAKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CABEÇALHO
* Nome do aluno: Emanuel Godinho Pedrozo
* Matrícula: 1415956 
* Professor: Victor Sales Silva
* Disciplina: PREPARAÇÃO E INGESTÃO DE DADOS
* Atividade: Exercício 3 - Coleta, tratamento e normalização de dados / Parte 2

INSTALAÇÃO BIBLIOTECA AZURE STOREGE BLOB

In [ ]:
pip install azure-storage-blob

In [ ]:
import requests
import zipfile
from azure.storage.blob import BlobClient
import os
import pandas as pd
import numpy as np

CADEIA DE CONEXÃO

In [ ]:
cadeia_conexao = 'DefaultEndpointsProtocol=https;AccountName=storageaccount1415956;AccountKey=VP3Anbpvb8xOCraYw0fllNTkc9nR/BfbmKObg+J1c+Q+lXkccsa2VZA/IO5Es7sel5HT0zKnIsOj+ASt+V2bqA==;EndpointSuffix=core.windows.net'

ENVIAR OS DADOS PARA AZURE STOREGE BLOB

In [ ]:
arquivo_blob = '/content/DADOS_EXAMES.csv'
blob = BlobClient.from_connection_string(conn_str=cadeia_conexao, container_name="datalake-aulas", blob_name='raw/DADOS_EXAMES/DADOS_EXAMES_2.csv' )
with open(arquivo_blob, "rb") as data:
  blob.upload_blob(data)  

URL DOS DADOS

In [ ]:
url_dados_exames = 'https://storageaccount1415956.blob.core.windows.net/datalake-aulas/raw/DADOS_EXAMES/DADOS_EXAMES.csv'

FAZENDO A LEITURA DOS DADOS

In [ ]:
df_url_dados_exames = pd.read_csv(url_dados_exames, sep=';')

In [ ]:
df_url_dados_exames.head()

,IdUnidadeAtendimento,NumPedido,IdExame,PrecoExame,DataPedido,QtdExames,QtdAmostrasColhidas,DataPrevistaEntregaResultado,DataLiberacaoResultado,IdCidade
0,0,PED-05738304,6986,"8,35",2022-06-03 07:58:00,1,0,2022-06-04 16:00:00,2022-06-03 18:32:07,1
1,0,PED-05746056,8397,"6,96",2022-06-03 08:07:00,1,0,2022-06-04 16:00:00,2022-06-03 16:05:56,1
2,0,PED-05752896,3376,"13,91",2022-06-03 08:13:00,1,1,2022-06-04 16:00:00,2022-06-03 16:38:30,1
3,0,PED-05757912,2142,"23,65",2022-06-03 08:19:00,1,1,2022-06-04 16:00:00,2022-06-03 14:36:31,1
4,0,PED-05763384,3272,"300,55",2022-06-03 08:28:00,2,2,2022-06-15 16:00:00,2022-06-06 15:42:11,1


AJUSTE DOS SCHEMAS

In [ ]:
df_url_dados_exames['IdExame'] = df_url_dados_exames['IdExame'].fillna(0)
df_url_dados_exames['QtdExames'] = df_url_dados_exames['QtdExames'].fillna(0)
df_url_dados_exames['QtdAmostrasColhidas'] = df_url_dados_exames['QtdAmostrasColhidas'].fillna(0)
df_url_dados_exames['IdExame'] = df_url_dados_exames['IdExame'].astype(int)
df_url_dados_exames['QtdExames'] = df_url_dados_exames['QtdExames'].astype(int)
df_url_dados_exames['QtdAmostrasColhidas'] = df_url_dados_exames['QtdAmostrasColhidas'].astype(int)
df_url_dados_exames['Cidade'] = df_url_dados_exames['Cidade'].astype('string')
df_url_dados_exames['Estado'] = df_url_dados_exames['Estado'].astype('string')
df_url_dados_exames['NumPedido'] = df_url_dados_exames['NumPedido'].astype('string')
df_url_dados_exames['Exame'] = df_url_dados_exames['Exame'].astype('string')
df_url_dados_exames['SiglaExame'] = df_url_dados_exames['SiglaExame'].astype('string')
df_url_dados_exames['Material'] = df_url_dados_exames['Material'].astype('string')
df_url_dados_exames['SetorExame'] = df_url_dados_exames['SetorExame'].astype('string')
df_url_dados_exames['SetorExame'] = df_url_dados_exames['SetorExame'].astype('string')
df_url_dados_exames['DataPedido'] = df_url_dados_exames['DataPedido'].astype('datetime64')
df_url_dados_exames['DataPrevistaEntregaResultado'] = df_url_dados_exames['DataPrevistaEntregaResultado'].astype('datetime64')
df_url_dados_exames['DataLiberacaoResultado'] = df_url_dados_exames['DataLiberacaoResultado'].astype('datetime64')

INSERÇÃO DO CAMPO IDCIDADE

In [ ]:
df_url_dados_exames['IdCidade'] = np.where(df_url_dados_exames['Cidade']=='BELO HORIZONTE', '1',
                                   np.where(df_url_dados_exames['Cidade']=='CONTAGEM', '2', ''))

FAZENDO A ORDENAÇÃO DE TODOS OS CAMPOS

In [ ]:
df_url_dados_exames[['IdUnidadeAtendimento', 'IdCidade', 'Cidade', 'Estado', 'NumPedido', 'IdExame', 'Exame', 'SiglaExame', 'Material', 'SetorExame', 'PrecoExame', 'DataPedido', 'QtdExames', 'QtdAmostrasColhidas', 'DataPrevistaEntregaResultado', 'DataLiberacaoResultado']]

AJUTES DO TIPO DO CAMPO IDCIDADE

In [ ]:
df_url_dados_exames['IdCidade'] = df_url_dados_exames['IdCidade'].astype('int64')

CRIAÇÃO DO DATRAME DIM_CIDADE

In [ ]:
df_dim_cidades = df_url_dados_exames.loc[:, ['IdCidade', 'Cidade', 'Estado']]

REMOÇÃO DAS INFORMAÇÃOES DUPLICADAS

In [ ]:
df_dim_cidades = df_dim_cidades.drop_duplicates()

In [ ]:
df_dim_cidades.head()

,IdCidade,Cidade,Estado
0,1,BELO HORIZONTE,MG
1029,2,CONTAGEM,MG


CRIAÇÃO DO DATAFRAME DIM_EXAME

In [ ]:
df_dim_exane = df_url_dados_exames.loc[:, ['IdExame', 'Exame', 'SiglaExame', 'Material', 'SetorExame']]

In [ ]:
df_dim_exane.head()

,IdExame,Exame,SiglaExame,Material,SetorExame
0,6986,TSH ULTRA SENSÍVEL,TSH-B,SANGUE,HORMONIO
1,8397,GAMA GT,YGT,SANGUE,BIOQUIMICA ESPECIALIZADA
2,3376,HIV 1 E 2 PESQUISA DE ANTÍGENO E ANTICORPOS,HIV-EL,SANGUE,IMUNOLOGIA
3,2142,COLESTEROL TOTAL E FRAÇÕES,ELFC,SANGUE,BIOQUIMICA ESPECIALIZADA
4,3272,ESTUDO GENÉTICO DAS MUTAÇÕES C282Y E H63D PARA...,HEMOCR,DIVERSOS,GEN. HUMANA


REMOÇÃO DOS CAMPO NO df_url_dados_exames

In [ ]:
df_url_dados_exames = df_url_dados_exames.drop('Cidade', axis=1)
df_url_dados_exames = df_url_dados_exames.drop('Estado', axis=1)
df_url_dados_exames = df_url_dados_exames.drop('Exame', axis=1)
df_url_dados_exames = df_url_dados_exames.drop('SiglaExame', axis=1)
df_url_dados_exames = df_url_dados_exames.drop('Material', axis=1)
df_url_dados_exames = df_url_dados_exames.drop('SetorExame', axis=1)

In [ ]:
df_url_dados_exames.head()

,IdUnidadeAtendimento,NumPedido,IdExame,PrecoExame,DataPedido,QtdExames,QtdAmostrasColhidas,DataPrevistaEntregaResultado,DataLiberacaoResultado,IdCidade
0,0,PED-05738304,6986,"8,35",2022-06-03 07:58:00,1,0,2022-06-04 16:00:00,2022-06-03 18:32:07,1
1,0,PED-05746056,8397,"6,96",2022-06-03 08:07:00,1,0,2022-06-04 16:00:00,2022-06-03 16:05:56,1
2,0,PED-05752896,3376,"13,91",2022-06-03 08:13:00,1,1,2022-06-04 16:00:00,2022-06-03 16:38:30,1
3,0,PED-05757912,2142,"23,65",2022-06-03 08:19:00,1,1,2022-06-04 16:00:00,2022-06-03 14:36:31,1
4,0,PED-05763384,3272,"300,55",2022-06-03 08:28:00,2,2,2022-06-15 16:00:00,2022-06-06 15:42:11,1
